In [1]:
import os
import mypackages.myrasters as mr
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data_dir = os.path.join('..', 'data/CGIAR/swc_fr')
irr_dir = os.path.join('..', 'output/irrigation_prepared')
out_dir = os.path.join('..', 'output/soil_water_content_prepared')

In [3]:
months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun',
          'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

Load ar raster to check properties:

In [4]:
jan_raster = mr.MyRaster(os.path.join(data_dir, 'swc_fr_1/hdr.adf'))

In [5]:
jan_raster.show_raster_properties()

Driver:  AIG
Projection:  GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]
----------------------------------------------------------------------------------------------------
Size:  43200 x 18000 x 1
X min =  -180.0
X max =  180.0000187754631
Y min =  -60.0
Y max =  90.00000782310963
Origin = ( -180.0 , 90.00000782310963 )
Pixel Size = ( 0.008333333767950535 , -0.008333333767950535 )
No Data Value =  255.0


Crop monthly arrays to size of soilgrid layers and staple them in an array:

In [6]:
%%time

swc_monthly = []
for i in range(0, 12):
    raster = mr.MyRaster(os.path.join(data_dir, 'swc_fr_' + str(i+1), 'hdr.adf'))
    array = mr.cut_array_yboundaries(raster.get_array(),  
                                     -60, 90.00000782310963, -56.00081039406058, 
                                     83.99916720600001, -0.008333333767950535)
    
    """
    # write rasters
    out_filename = str(i+1) + '_swc_' + months[i]
    mr.array2geotiff(array, os.path.join(out_dir, out_filename),
                     raster.NoDataValue, array.shape[1], array.shape[0], 
                     -180, 83.99916720600001, raster.pixelWidth, raster.pixelHeight)
    """
    
    swc_monthly.append(array)
swc_monthly = np.array(swc_monthly)

CPU times: user 39 s, sys: 10.2 s, total: 49.2 s
Wall time: 49 s


Transform arrays to a 1 to 5 scale:

In [7]:
%%time

swc_scaled_monthly = []

for i in range(0, 12):
    scaled = swc_monthly[i] / 2.5 + 10
    scaled[swc_monthly[i] == 255] = 255
    array = scaled.round(0)
    array = array.astype('uint8')
    
    """
    # write rasters
    out_filename = str(i+1) + '_swc_scaled_' + months[i]
    mr.array2geotiff(array, os.path.join(out_dir, out_filename),
                     raster.NoDataValue, array.shape[1], array.shape[0], 
                     -180, 83.99916720600001, raster.pixelWidth, raster.pixelHeight)
    """
    
    swc_scaled_monthly.append(array)

swc_scaled_monthly = np.array(swc_scaled_monthly) 

CPU times: user 1min 37s, sys: 36.8 s, total: 2min 14s
Wall time: 2min 14s


In [8]:
"""
# read rasters
swc_scaled_monthly = []
for i in range(0, 12):
    out_filename = str(i+1) + '_swc_scaled_' + months[i] +'.tif'
    raster = mr.MyRaster(os.path.join(out_dir, out_filename))
    array = raster.get_array()
    swc_scaled_monthly.append(array)
swc_scaled_monthly = np.array(swc_monthly)
"""

"\n# read rasters\nswc_scaled_monthly = []\nfor i in range(0, 12):\n    out_filename = str(i+1) + '_swc_scaled_' + months[i] +'.tif'\n    raster = mr.MyRaster(os.path.join(out_dir, out_filename))\n    array = raster.get_array()\n    swc_scaled_monthly.append(array)\nswc_scaled_monthly = np.array(swc_monthly)\n"

Load irrigation data:

In [9]:
aai_raster = mr.MyRaster(os.path.join(irr_dir, 'aai_resampled.tif'))
aai = aai_raster.get_array()
aai[aai == aai_raster.NoDataValue] = 0

Correct soil moisture values where they are smaller than 30 assuming that irrigated areas get to a soil moisture value of 30:

In [10]:
%%time

irr_share = aai/100  # share of irrigated area
irr = irr_share * 30  # irrigated areas are set to soil moisture category 30
non_irr_share = 1 - aai/100  # share of non-irrigated areas

for i in range(0, 12):
    swc = swc_scaled_monthly[i]
    corr = irr + non_irr_share * swc  # calculate correction layer
    corr[corr > 50] = 255
    corr = corr.round(0)
    corr = corr.astype('uint8')
    
    swc[swc < 30] = corr[swc < 30]  # choose corrected values where swc < 30
    swc[corr == 255] = 255
    
    mr.array2geotiff(swc, os.path.join(out_dir, str(i+1) + '_swc_corr_' + months[i]),
                     255, swc.shape[1], swc.shape[0],
                     -180, 83.99916720600001, aai_raster.pixelWidth, aai_raster.pixelHeight)

CPU times: user 2min 32s, sys: 31.9 s, total: 3min 4s
Wall time: 3min 4s
